In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math
import torch
import torch.nn as nn
from torch import functional as F
from scipy import integrate
from sympy import *
import torch.optim as optim

%matplotlib inline
%config InlineBackend.figure_format = 'retina'



np.random.seed(1234)


In [2]:
pd.set_option('display.max_rows', 1000)
pd.set_option('display.max_columns', 1000)

# データセットを作る
まずコードが動くかどうか確認したいので、100個ぐらいでOK


# ブラウン運動の発生

In [3]:
def brownian_motion(n, T):
    """
    Simulates a Brownian motion
    :param int n : the number of discrete steps
    :param int T: the number of continuous time steps
    :param float h: the variance of the increments
    """
    delta_t = 1. * T/n  # decide the increment of the time
    partition = [i * delta_t for i in range(n + 1)] # make a partition
    
    # ブラウン運動の差分（平均：０，標準偏差：時間の差分）
    random_increments = np.random.normal(loc = 0.0, scale = np.sqrt(delta_t), size = n)
    '''
    where loc = "mean(average)", scale = "variance", n = the number of increments.
    (正規分布を発生させている)
    '''
    # making data like a Brownian motion
    brownian_motion = np.cumsum(random_increments)  # calculate the brownian motion
    # insert the initial condition
    brownian_motion = np.insert(brownian_motion, 0, 0.0)
    
    return brownian_motion, random_increments, partition

# O-Uモデル（模擬データの作成）

In [4]:
def simulate_OU(n = 100, T = 1, mu = 3, X_0 = 0.1, sigma = 0.1, fig_mode = False):
    # BMの生成
    BM_1, dB_1, partition = brownian_motion(n, T)
    dt = 1. * T / n
    
    # O-U model に従う ”Process X”を作成
    X = np.zeros(n + 1)

    X[0] = X_0
    for i, dB_1_t, t in zip(range(1, n+1), dB_1, partition):
        # 1つ前の Xの値
        pre_X = X[i-1] 
        # X の値を計算（SDEに従う）
        X[i] = pre_X + sigma * dB_1_t  -  mu * pre_X * dt
        
    
    data_array = np.array([partition, X]).T
    df = pd.DataFrame(data_array, columns = ['timestamp', 'process'])
    
    if fig_mode:
        fig, ax = plt.subplots()

        # plot the process X and volatility sigma
        ax.plot(partition, X, color = 'blue', label = 'process')
        ax.set_xlabel('time(s)')
        ax.set_ylabel('process X')

        # 以下はそんなに関係ないから気にしなくていい．
        plt.gca().spines['right'].set_visible(False)
        plt.gca().spines['top'].set_visible(False)
        plt.gca().yaxis.set_ticks_position('left')
        plt.gca().xaxis.set_ticks_position('bottom')

        plt.legend()
    
    return df


# ハイパーパラメータの設定

In [5]:
#時間
n = 100
#満期
T = 1 
#リスクフリーレート
r = 0.001
#行使価格
K = 900

## 初期値の設定

In [6]:
X_0 = 1
mu = 0
sigma = 1

In [8]:
#pathの各payoffを計算
datas_Payoff = []
for t, x in zip(df["timestamp"], df["process"]):
  data = math.exp(- r * t) * max( ( K - x ) , 0)
  datas_Payoff.append(data)

In [9]:
max(datas_Payoff)

0.0

#データの作成

In [10]:
#データ数
N = 10**2

In [11]:
DATAs_input = []
for _ in range(N):
    df_path = simulate_OU(n = n, T = T, mu = mu, X_0 = X_0, sigma = sigma, fig_mode = False)
    partition = df_path["timestamp"].values
    X = df_path["process"].values
    DATAs_input.append(df_path.values)
DATAs_input = np.array(DATAs_input)

In [12]:
DATAs_teacher = []
for _ in range(N):
    df = simulate_OU(n = n, T = T, mu = mu, X_0 = X_0, sigma = sigma, fig_mode = False)
    datas_Payoff = []
    for t, x in zip(df["timestamp"], df["process"]):
      data = math.exp(- r * t) * max( (x - K) , 0)
      datas_Payoff.append(data)
    Phi = max(datas_Payoff)
    DATAs_teacher.append(Phi)
DATAs_teacher = np.array(DATAs_teacher)

In [13]:
DATAs_input = torch.Tensor(DATAs_input)
DATAs_teacher = torch.Tensor(DATAs_teacher)

## ニューラルネットワーク層を作る
まずは1層か2層でOK

In [14]:
fc = nn.Linear(1,2)

#モデルの作成

In [15]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Linear( 2, 1)

    def forward(self, x):
        # フォワードパスを定義
        x = torch.sigmoid(self.layer1(x))  
        return x

In [16]:
# モデル（NeuralNetworkクラス）のインスタンス化
model = NeuralNetwork()

In [17]:
y = model(DATAs_input) 

## 誤差関数を定義

今定義したいのは、
$$\int_a^b (|\Phi(x)-U(x)|^2) dx$$

In [18]:
class CustomLoss(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, targets, outputs):
        loss = (targets - outputs) ** 2
        #inputs = symbols("inputs")
        #loss = integrate(integ, (inputs, a, b))
        #loss = integ
        return loss

In [19]:
criterion = CustomLoss()
inputs = DATAs_input
outputs = model(inputs)
targets = DATAs_teacher

In [20]:
a = 0
b = 1

In [21]:
loss = criterion(outputs, targets)

In [22]:
loss_sum = torch.sum(loss)

In [23]:
loss_sum.backward()

#最適化

In [24]:
optimizer = optim.SGD(model.parameters(), lr = 0.1)

#学習

In [25]:
epochs = 100

In [26]:
model.train()
for epoch in range(epochs):
  optimizer.zero_grad()
  outputs = model(inputs)
  loss = criterion(outputs, targets)
  loss_sum = torch.sum(loss)
  loss_sum.backward()
  optimizer.step()

In [27]:
model(DATAs_input)

tensor([[[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        ...,

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]],

        [[1.],
         [1.],
         [1.],
         ...,
         [1.],
         [1.],
         [1.]]], grad_fn=<SigmoidBackward>)

In [28]:
loss_fn = CustomLoss()

In [29]:
loss_fn(model(DATAs_input),DATAs_teacher)
sum_loss_fn = torch.sum(loss_fn(model(DATAs_input),DATAs_teacher))
sum_loss_fn

tensor(1.0031e+10, grad_fn=<SumBackward0>)

# メモ